In [100]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow import keras 
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential 
from keras import Input 
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import LSTM
from sklearn.metrics import roc_curve,auc,roc_auc_score

In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
data=pd.read_csv("/content/drive/MyDrive/AIProject/taiyo/Data.csv")
data

,Date,Open,High,Low,Close,Volume,Currency,Name,Country
0,2020-01-01,128.75,128.90,126.85,127.45,2574894,INR,ONGC,India
1,2020-01-02,127.65,128.65,127.10,128.05,4759843,INR,ONGC,India
2,2020-01-03,131.00,133.40,128.05,128.45,31381588,INR,ONGC,India
3,2020-01-06,129.70,129.80,125.10,126.25,14611204,INR,ONGC,India
4,2020-01-07,125.60,127.70,125.40,125.75,7193304,INR,ONGC,India
...,...,...,...,...,...,...,...,...,...
3056,2022-06-10,282.80,285.35,279.00,280.10,7245446,INR,ADAN,India
3057,2022-06-13,269.50,275.50,266.10,266.20,11186749,INR,ADAN,India
3058,2022-06-14,261.60,279.50,261.60,266.65,16393298,INR,ADAN,India
3059,2022-06-15,270.00,275.70,270.00,272.05,7446565,INR,ADAN,India


In [103]:
data=data.drop(['Currency','Country'],axis=1)
data.head()

,Date,Open,High,Low,Close,Volume,Name
0,2020-01-01,128.75,128.90,126.85,127.45,2574894,ONGC
1,2020-01-02,127.65,128.65,127.10,128.05,4759843,ONGC
2,2020-01-03,131.00,133.40,128.05,128.45,31381588,ONGC
3,2020-01-06,129.70,129.80,125.10,126.25,14611204,ONGC
4,2020-01-07,125.60,127.70,125.40,125.75,7193304,ONGC


In [104]:
data.set_index('Date',drop=True,inplace=True)

In [105]:
data

,Open,High,Low,Close,Volume,Name
Date,,,,,,
2020-01-01,128.75,128.90,126.85,127.45,2574894,ONGC
2020-01-02,127.65,128.65,127.10,128.05,4759843,ONGC
2020-01-03,131.00,133.40,128.05,128.45,31381588,ONGC
2020-01-06,129.70,129.80,125.10,126.25,14611204,ONGC
2020-01-07,125.60,127.70,125.40,125.75,7193304,ONGC
...,...,...,...,...,...,...
2022-06-10,282.80,285.35,279.00,280.10,7245446,ADAN
2022-06-13,269.50,275.50,266.10,266.20,11186749,ADAN
2022-06-14,261.60,279.50,261.60,266.65,16393298,ADAN


In [106]:
data['Name']=data['Name'].replace("ONGC",1)
data['Name']=data['Name'].replace("BPCL",2)
data['Name']=data['Name'].replace("HPCL",3)
data['Name']=data['Name'].replace("GAIL",4)
data['Name']=data['Name'].replace("ADAN",5)

In [107]:
y=data['Close']
X=data.drop('Close',axis=1)

In [108]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [109]:
Mxs=MinMaxScaler()
X_train=Mxs.fit_transform(X_train)
X_test=Mxs.fit_transform(X_test)

In [110]:
X_train.shape[1]

5

In [132]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape = (5,1)))
model.add(Dense(units=25,activation='relu',name="hidden-layer"))
model.add(LSTM(units=50))
model.add(Dense(units=25,activation='relu',name="hidden-layer-2"))
model.add(Dense(units=25,activation='relu',name="hidden-layer-3"))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_29 (LSTM)              (None, 5, 50)             10400     
                                                                 
 hidden-layer (Dense)        (None, 5, 25)             1275      
                                                                 
 lstm_30 (LSTM)              (None, 50)                15200     
                                                                 
 hidden-layer-2 (Dense)      (None, 25)                1275      
                                                                 
 hidden-layer-3 (Dense)      (None, 25)                650       
                                                                 
 dense_20 (Dense)            (None, 1)                 26        
                                                                 
Total params: 28,826
Trainable params: 28,826
Non-tra

In [133]:
model.fit(X_train, y_train, epochs=100, verbose=1)

Epoch 1/100
67/67 [==============================] - 5s 15ms/step - loss: 46885.3633 - mean_absolute_error: 181.0007
Epoch 2/100
67/67 [==============================] - 1s 10ms/step - loss: 23830.2207 - mean_absolute_error: 114.3579
Epoch 3/100
67/67 [==============================] - 1s 8ms/step - loss: 14062.9756 - mean_absolute_error: 100.4550
Epoch 4/100
67/67 [==============================] - 1s 9ms/step - loss: 5285.9038 - mean_absolute_error: 45.5958
Epoch 5/100
67/67 [==============================] - 1s 9ms/step - loss: 705.5273 - mean_absolute_error: 16.6623
Epoch 6/100
67/67 [==============================] - 1s 8ms/step - loss: 194.4422 - mean_absolute_error: 9.4558
Epoch 7/100
67/67 [==============================] - 1s 8ms/step - loss: 82.2685 - mean_absolute_error: 6.3032
Epoch 8/100
67/67 [==============================] - 1s 9ms/step - loss: 41.4636 - mean_absolute_error: 4.2873
Epoch 9/100
67/67 [==============================] - 1s 8ms/step - loss: 27.1771 - mean_a

In [134]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [136]:
y_train=np.array(y_train)
y_test=np.array(y_test)

## METRICS

In [138]:
print("r2 score over TRAIN data: "+str(r2_score(y_train,pred_train)))
print("r2 score over TEST data: "+str(r2_score(y_test,pred_test)))

r2 score over TRAIN data: 0.9987719597085768
r2 score over TEST data: 0.9978567444018928


In [139]:
print("MEAN SQUARED ERROR score over TRAIN data: "+str(mean_squared_error(y_train,pred_train)))
print("MEAN SQUARED ERROR score over TEST data: "+str(mean_squared_error(y_test,pred_test)))

MEAN SQUARED ERROR score over TRAIN data: 17.31584521960877
MEAN SQUARED ERROR score over TEST data: 29.389516563569682
